In [30]:
from pathlib import Path
import requests
import pandas as pd
import numpy as np
from io import StringIO
from simple_salesforce import Salesforce
import shutil
import datetime
from datetime import date


sf = Salesforce(
    username="jmartinez@careassistance.com",
    password="Jcmm01041983*",
    security_token="dxbNevVOmgmoQS4Zllh3DpNDj",
)

In [31]:
results = sf.query_all(
    """
    select Id, IdentificationId__pc from Account where ispersonaccount = True
   """
)
Pacientes_SF = pd.DataFrame(results["records"])

Pacientes_SF = Pacientes_SF[
    [
        "Id",
        "IdentificationId__pc",
    ]
]
Pacientes_SF = Pacientes_SF.rename(
    columns={
        "Id": "Id Account Paciente",
        "IdentificationId__pc": "Rut",
    }
)
Pacientes_SF.shape



(1807375, 2)

In [32]:
results = sf.query_all(
    """
    select id,IdentificationId__c from Contact 
   """
)
Pacientes_SF2 = pd.DataFrame(results["records"])

Pacientes_SF2 = Pacientes_SF2[
    [
        "Id",
        "IdentificationId__c",
    ]
]
Pacientes_SF2 = Pacientes_SF2.rename(
    columns={
        "Id": "Id Contact Paciente",
        "IdentificationId__c": "Rut",
    }
)
Pacientes_SF2.shape

(1807375, 2)

In [33]:
Pacientes_SF2 = pd.merge(
    left=Pacientes_SF,
    right=Pacientes_SF2,
    how="left",
    left_on="Rut",
    right_on="Rut",
)
Pacientes_SF2.shape

(1807375, 3)

In [34]:
Pacientes_SF2.head()

,Id Account Paciente,Rut,Id Contact Paciente
0,0018c00002amLTMAA2,173805004,0038c000035CV93AAG
1,0018c00002amLUGAA2,153338949,0038c000035CV94AAG
2,0018c00002amLUHAA2,177735213,0038c000035CV95AAG
3,0018c00002amLUIAA2,169332924,0038c000035CV9zAAG
4,0018c00002amLUJAA2,172957692,0038c000035CVA0AAO


In [35]:
results = sf.query_all(
    """
    select id,Paciente__c,Programas_por_Empresas__c from Paciente_Programas__c 
   """
)
Paciente_Programa = pd.DataFrame(results["records"])

Paciente_Programa = Paciente_Programa[
    [
        "Paciente__c",
        "Programas_por_Empresas__c",
    ]
]
Paciente_Programa = Paciente_Programa.rename(
    columns={
        "Paciente__c": "Id paciente account",
        "Programas_por_Empresas__c": "Programas por empresa",
    }
)
Paciente_Programa.shape

(1415656, 2)

In [36]:
results = sf.query_all(
    """
    select Id,Empresa__c from Programas_por_Empresas__c  
   """
)
empresas = pd.DataFrame(results["records"])

empresas = empresas[
    [
        "Id",
        "Empresa__c",
    ]
]
empresas = empresas.rename(
    columns={
        "Id": "Programas_por_Empresas__c",
        "Empresa__c": "Id Account Empresa",
    }
)
empresas.shape




(7260, 2)

In [37]:
empresas.shape

(7260, 2)

In [38]:
Paciente_Programa = pd.merge(
    left=Paciente_Programa,
    right=empresas,
    how="left",
    left_on="Programas por empresa",
    right_on="Programas_por_Empresas__c",
)
Paciente_Programa.shape

(1415656, 4)

In [39]:
Paciente_Programa.head()

,Id paciente account,Programas por empresa,Programas_por_Empresas__c,Id Account Empresa
0,0018c00002amLbpAAE,a1W8c000007vDPvEAM,a1W8c000007vDPvEAM,0018c00002axQaeAAE
1,0018c00002amLbqAAE,a1W8c000007vDSpEAM,a1W8c000007vDSpEAM,0018c00002axQecAAE
2,0018c00002amLbrAAE,a1W8c000007vDSpEAM,a1W8c000007vDSpEAM,0018c00002axQecAAE
3,0018c00002amLbsAAE,a1W8c000007vDSpEAM,a1W8c000007vDSpEAM,0018c00002axQecAAE
4,0018c00002amLc2AAE,a1W8c000007vDSpEAM,a1W8c000007vDSpEAM,0018c00002axQecAAE


In [40]:
results = sf.query_all(
    """
    select AccountId, ContactId from AccountContactRelation 
   """
)
accountcontact = pd.DataFrame(results["records"])

accountcontact = accountcontact[
    [
        "AccountId",
        "ContactId",
    ]
]
accountcontact = accountcontact.rename(
    columns={
        "AccountId": "Id account",
        "ContactId": "id contacto",
    }
)
accountcontact.shape

(1549869, 2)

In [41]:
accountcontact['LLave'] = accountcontact['id contacto'] + accountcontact['Id account'] 
accountcontact.head()

,Id account,id contacto,LLave
0,0018c00002axQdlAAE,0038c000036jelbAAA,0038c000036jelbAAA0018c00002axQdlAAE
1,0018c00002axQdlAAE,0038c000035CWLYAA4,0038c000035CWLYAA40018c00002axQdlAAE
2,0018c00002axQdlAAE,0038c000035CaxiAAC,0038c000035CaxiAAC0018c00002axQdlAAE
3,0018c00002axQdlAAE,0038c000036jelcAAA,0038c000036jelcAAA0018c00002axQdlAAE
4,0018c00002axQa5AAE,0038c000036jLDoAAM,0038c000036jLDoAAM0018c00002axQa5AAE


In [42]:
accountcontact.shape

(1549869, 3)

In [43]:
Paciente_Programa.head()

,Id paciente account,Programas por empresa,Programas_por_Empresas__c,Id Account Empresa
0,0018c00002amLbpAAE,a1W8c000007vDPvEAM,a1W8c000007vDPvEAM,0018c00002axQaeAAE
1,0018c00002amLbqAAE,a1W8c000007vDSpEAM,a1W8c000007vDSpEAM,0018c00002axQecAAE
2,0018c00002amLbrAAE,a1W8c000007vDSpEAM,a1W8c000007vDSpEAM,0018c00002axQecAAE
3,0018c00002amLbsAAE,a1W8c000007vDSpEAM,a1W8c000007vDSpEAM,0018c00002axQecAAE
4,0018c00002amLc2AAE,a1W8c000007vDSpEAM,a1W8c000007vDSpEAM,0018c00002axQecAAE


In [44]:
Paciente_Programa.shape

(1415656, 4)

In [45]:
Pacientes_SF2.head()

,Id Account Paciente,Rut,Id Contact Paciente
0,0018c00002amLTMAA2,173805004,0038c000035CV93AAG
1,0018c00002amLUGAA2,153338949,0038c000035CV94AAG
2,0018c00002amLUHAA2,177735213,0038c000035CV95AAG
3,0018c00002amLUIAA2,169332924,0038c000035CV9zAAG
4,0018c00002amLUJAA2,172957692,0038c000035CVA0AAO


In [46]:
Pacientes_SF2.shape

(1807375, 3)

In [47]:
programa_paciente = pd.merge(
    left=Paciente_Programa,
    right=Pacientes_SF2,
    how="left",
    left_on="Id paciente account",
    right_on="Id Account Paciente",
)
programa_paciente.shape

(1415656, 7)

In [48]:
programa_paciente.head()

,Id paciente account,Programas por empresa,Programas_por_Empresas__c,Id Account Empresa,Id Account Paciente,Rut,Id Contact Paciente
0,0018c00002amLbpAAE,a1W8c000007vDPvEAM,a1W8c000007vDPvEAM,0018c00002axQaeAAE,0018c00002amLbpAAE,224488556,0038c000035CVHVAA4
1,0018c00002amLbqAAE,a1W8c000007vDSpEAM,a1W8c000007vDSpEAM,0018c00002axQecAAE,0018c00002amLbqAAE,113376066,0038c000035CVHWAA4
2,0018c00002amLbrAAE,a1W8c000007vDSpEAM,a1W8c000007vDSpEAM,0018c00002axQecAAE,0018c00002amLbrAAE,79476161,0038c000035CVHXAA4
3,0018c00002amLbsAAE,a1W8c000007vDSpEAM,a1W8c000007vDSpEAM,0018c00002axQecAAE,0018c00002amLbsAAE,181854227,0038c000035CVHYAA4
4,0018c00002amLc2AAE,a1W8c000007vDSpEAM,a1W8c000007vDSpEAM,0018c00002axQecAAE,0018c00002amLc2AAE,126899823,0038c000035CVHnAAO


In [49]:
programa_paciente.shape

(1415656, 7)

In [50]:
programa_paciente = programa_paciente[
    [
        "Id Contact Paciente",
        "Id Account Empresa",
    ]
]
programa_paciente['LLave'] = programa_paciente['Id Contact Paciente'] + programa_paciente['Id Account Empresa']
programa_paciente = programa_paciente.rename(
    columns={
        "Id Contact Paciente": "ContactId",
        "Id Account Empresa": "AccountId",
    }
)
programa_paciente.head()

,ContactId,AccountId,LLave
0,0038c000035CVHVAA4,0018c00002axQaeAAE,0038c000035CVHVAA40018c00002axQaeAAE
1,0038c000035CVHWAA4,0018c00002axQecAAE,0038c000035CVHWAA40018c00002axQecAAE
2,0038c000035CVHXAA4,0018c00002axQecAAE,0038c000035CVHXAA40018c00002axQecAAE
3,0038c000035CVHYAA4,0018c00002axQecAAE,0038c000035CVHYAA40018c00002axQecAAE
4,0038c000035CVHnAAO,0018c00002axQecAAE,0038c000035CVHnAAO0018c00002axQecAAE


In [51]:
programa_paciente.shape

(1415656, 3)

In [52]:
programa_paciente["RutEstaoNo"] = programa_paciente["LLave"].isin(accountcontact['LLave'])
programa_paciente.shape

(1415656, 4)

In [53]:
programa_paciente.head()


,ContactId,AccountId,LLave,RutEstaoNo
0,0038c000035CVHVAA4,0018c00002axQaeAAE,0038c000035CVHVAA40018c00002axQaeAAE,True
1,0038c000035CVHWAA4,0018c00002axQecAAE,0038c000035CVHWAA40018c00002axQecAAE,True
2,0038c000035CVHXAA4,0018c00002axQecAAE,0038c000035CVHXAA40018c00002axQecAAE,True
3,0038c000035CVHYAA4,0018c00002axQecAAE,0038c000035CVHYAA40018c00002axQecAAE,True
4,0038c000035CVHnAAO,0018c00002axQecAAE,0038c000035CVHnAAO0018c00002axQecAAE,True


In [54]:
programa_paciente1 = programa_paciente[programa_paciente["RutEstaoNo"] == False] 

In [55]:
accountcontact["RutEstaoNo"] = accountcontact["LLave"].isin(programa_paciente['LLave'])
accountcontact1 = accountcontact[accountcontact["RutEstaoNo"] == False]
accountcontact1.head()


,Id account,id contacto,LLave,RutEstaoNo
23,0018c00002axQa5AAE,0038c000036jMBzAAM,0038c000036jMBzAAM0018c00002axQa5AAE,False
58,0018c00002axQdeAAE,0038c000036jPGcAAM,0038c000036jPGcAAM0018c00002axQdeAAE,False
149,0018c00002axQdcAAE,0038c000036izTKAAY,0038c000036izTKAAY0018c00002axQdcAAE,False
201,0018c00002axQdcAAE,0038c000036izU9AAI,0038c000036izU9AAI0018c00002axQdcAAE,False
538,0018c00002axQdcAAE,0038c000036izlMAAQ,0038c000036izlMAAQ0018c00002axQdcAAE,False


In [56]:
accountcontact.shape

(1549869, 4)

In [57]:
programa_paciente1.shape

(4027, 4)

In [58]:
programa_paciente1.to_excel("F:/Empresas SF/reportes/accountcontact.xlsx", index=False)
accountcontact1.to_excel("F:/Empresas SF/reportes/accountcontact1.xlsx", index=False)